In [1]:
import os

In [2]:
%pwd

'e:\\reviewprediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\reviewprediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: 300
    min_child_weight: 4
    max_depth: 5 
    learning_rate: 0.1
    target_column: str

In [6]:
from reviewprediction.constants import *
from reviewprediction.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.XGBClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            min_child_weight = params.min_child_weight,
            max_depth = params.max_depth,
            learning_rate = params.learning_rate,
            target_column = schema.name
            
        )

        return model_trainer_config

In [16]:
import pandas as pd
import os
from reviewprediction import logger
from xgboost import XGBClassifier
import json
import joblib
import numpy as np


In [14]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):


        # Define the file paths (replace with your actual paths)
        train_data = self.config.train_data_path
        test_data= self.config.test_data_path

        # Load the JSON data
        try:
            with open(train_data, "r") as f:
                train_data = json.load(f)
            with open(test_data, "r") as f:
                test_data = json.load(f)
        except FileNotFoundError:
            print("Error: JSON files not found at specified paths.")
        else:
            print("Train and test data loaded from JSON files.")

        # Extract features and labels
        X_train_final = []
        y_train = []
        for item in train_data:
            X_train_final.append(item["features"])
            y_train.append(item["label"])

        X_test_final = []
        y_test = []
        for item in test_data:
            X_test_final.append(item["features"])
            y_test = [item["label"] for item in test_data]  # Shortened list comprehension

        # Convert features back to NumPy arrays (if needed)
        X_train = np.array(X_train_final)
        X_test = np.array(X_test_final)

        # Define and train the XGBClassifier model
        model_XGB = XGBClassifier(n_estimators=self.config.n_estimators, min_child_weight=self.config.min_child_weight, max_depth=self.config.max_depth, learning_rate=self.config.learning_rate)
        model_XGB.fit(X_train, y_train)


        joblib.dump(model_XGB, os.path.join(self.config.root_dir, self.config.model_name))

In [15]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-03-07 20:08:53,511: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-07 20:08:53,556: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-07 20:08:53,571: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-07 20:08:53,582: INFO: common: created directory at: artifacts]
[2024-03-07 20:08:53,584: INFO: common: created directory at: artifacts/model_trainer]
Train and test data loaded from JSON files.
